# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`.

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [156]:
!python -m pip install semantic-kernel==0.3.14.dev0

27885.54s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [157]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI backend used by the kernel
if useAzureOpenAI:
    
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-4", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

## It all begins with an ask

In [158]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [160]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

# Basic Planner

 Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [162]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [128]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [163]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


You can see that the Planner:
- took the ask and converted it into an JSON-based plan 
- detailed how the AI would go about solving this task
- used the skills that the Kernel has available to it

The AI has determined which functions to call in order to fulfill the user ask.
The output of each step of the plan becomes the input to the next function.

Let's also define an inline skill and have it be available to the Planner. Be sure to give it a function name and skill name.

In [164]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill

In [165]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style."""

new_plan = await planner.create_plan_async(ask, kernel)

In [166]:
print(new_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}}
    ]
}


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [133]:
results = await planner.execute_plan_async(new_plan, kernel)

In [134]:
print(results)

1. A candlelit eve, bathed in romantic glow,
   At a restaurant of grandeur, hearts aflutter shall go.
   
2. In a verdant meadow, by soft breeze kissed,
   A picnic of heart-shaped sandwiches and champagne, we'll persist.
   
3. A day of spa, where bliss shall dwell,
   Together, we'll relish, in pampering, our souls to sell.
   
4. A cooking class, where love's flavors shall arise,
   A romantic dinner at home, we'll craft with passion in our eyes.
   
5. Ascend a scenic trail, hand in hand,
   At the summit, a picnic awaits, amidst nature's grandstand.
   
6. A surprise date night, with a limo's embrace,
   To a show or concert, we'll journey, wrapped in love's grace.
   
7. A night at home, with movies of ardor's tale,
   Homemade popcorn in hand, hearts entwined, we won't fail.
   
8. A dance class, where steps of romance we'll learn,
   Salsa or tango, our bodies entangled, love's fire shall burn.
   
9. A weekend's escape, to a mountain cabin we'll wend,
   Cozy and secluded, lo

# The Plan Object Model

To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# Sequential Planner

The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)

In [135]:
from semantic_kernel.planning import SequentialPlanner
planner = SequentialPlanner(kernel)

In [136]:
sequential_plan = await planner.create_plan_async(goal=ask)

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions

In [137]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Given a goal or topic description generate a list of ideas : {'variables': {'input': ''}}
Generic function, unknown purpose : {'variables': {'input': ''}}


Let's ask the sequential planner to execute the plan.

In [139]:
result = await sequential_plan.invoke_async()

In [140]:
print(result)

1. Verily, a romantic candlelit dinner at a fancy eatery
2. A picnic in the park, with heart-shaped bread and sweetmeats
3. Forsooth, a day of indulgence at the spa, where couples are embraced and massaged
4. Ascending a scenic trail, and partaking of a repast at its summit
5. Gliding upon the icy surface, ice skates adorning our feet, and partaking of warm cocoa thereafter
6. A night of cinema within the confines of our home, where romantic tales unfurl
7. Embarking on a voyage through vineyards, sampling wines and savoring each sip
8. Engaging in culinary instruction, together crafting a sumptuous feast
9. Sailing upon a boat during the twilight hour, beholding the wondrous vista
10. A night of mirth and games, seated at our humble abode, partaking of delectable treats and engaging in board games.


# Action Planner

The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.

In [141]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)

Let's add more skills to the kernel

In [142]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

{'lowercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x127f5add0>,
 'trim': <semantic_kernel.orchestration.sk_function.SKFunction at 0x127f5ae00>,
 'trim_end': <semantic_kernel.orchestration.sk_function.SKFunction at 0x127f5ae90>,
 'trim_start': <semantic_kernel.orchestration.sk_function.SKFunction at 0x127f5af20>,
 'uppercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x127f5afb0>}

In [143]:
ask = "What is the sum of 110 and 990?"

In [144]:
plan = await planner.create_plan_async(goal=ask)

In [145]:
result = await plan.invoke_async()

In [146]:
print(result)

1100


# Stepwise Planner

Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)

In [147]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

Let's create a Bing Search native skill that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)

In [148]:
class WebSearchEngineSkill:
    """
    A search engine skill.
    """
    from semantic_kernel.orchestration.sk_context import SKContext
    from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter

    def __init__(self, connector) -> None:
        self._connector = connector

    @sk_function(
        description="Performs a web search for a given query", name="searchAsync"
    )
    @sk_function_context_parameter(
        name="query",
        description="The search query",
    )
    async def search_async(self, query: str, context: SKContext) -> str:
        query = query or context.variables.get("query")[1]
        result = await self._connector.search_async(query, num_results=5, offset=0)
        return str(result)

In [149]:
from semantic_kernel.connectors.search_engine import BingConnector

BING_API_KEY = sk.bing_search_settings_from_dot_env()
connector = BingConnector(BING_API_KEY)
kernel.import_skill(WebSearchEngineSkill(connector), skill_name="WebSearch")

{'searchAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x127f23f10>}

Let's also add a couple more skills

In [150]:
from semantic_kernel.core_skills.math_skill import MathSkill
from semantic_kernel.core_skills.time_skill import TimeSkill

kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(MathSkill(), "math")

{'Add': <semantic_kernel.orchestration.sk_function.SKFunction at 0x127f58c70>,
 'Subtract': <semantic_kernel.orchestration.sk_function.SKFunction at 0x127f5ac20>}

In [151]:
planner = StepwisePlanner(
    kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000)
)

Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.

In [152]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style."""

plan = planner.create_plan(goal=ask)

In [153]:
result = await plan.invoke_async()
print(result)

Here are a few Shakespearean-style date ideas for Valentine's day:

1. A fair and merry picnic in yon park
2. Preparing a feast as one, in our humble abode
3. Embarking on a nature's voyage, a hike to explore
4. Venturing to a novel eatery, to taste foreign delights
5. Paying homage to local culture, within a museum's halls
6. Witnessing jesters and performers on stage, merry laughter abound
7. Whirling and twirling, in a dance class of shared waltz
8. Mounting noble steeds, or a tandem chariot of two wheels
9. Reveling in cinema, a homemade spectacle of moving pictures
10. Journeying through scenic realms, to a nearby hamlet or sandy shore.

These ideas should make for a delightful and memorable Valentine's day celebration!


In [154]:
print(result)

Here are a few Shakespearean-style date ideas for Valentine's day:

1. A fair and merry picnic in yon park
2. Preparing a feast as one, in our humble abode
3. Embarking on a nature's voyage, a hike to explore
4. Venturing to a novel eatery, to taste foreign delights
5. Paying homage to local culture, within a museum's halls
6. Witnessing jesters and performers on stage, merry laughter abound
7. Whirling and twirling, in a dance class of shared waltz
8. Mounting noble steeds, or a tandem chariot of two wheels
9. Reveling in cinema, a homemade spectacle of moving pictures
10. Journeying through scenic realms, to a nearby hamlet or sandy shore.

These ideas should make for a delightful and memorable Valentine's day celebration!


Let's see the steps that the AI took to get to the answer.

In [155]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  To come up with date ideas, I can use the WriterSkill.Brainstorm function to generate a list of ideas. Then, I can use the ShakespeareSkill.shakespeare function to write the ideas in Shakespearean style.
  [ACTION]
  {"action": "WriterSkill.Brainstorm", "action_variables": {"input": "date ideas"}}
  [OBSERVATION]
  1. Picnic in the park
  2. Cooking a meal together at home
  3. Going for a hike and exploring nature
  4. Trying out a new restaurant or cuisine
  5. Visiting a local museum or art gallery
  6. Going to a comedy show or live performance
  7. Taking a dance class together
  8. Going on a bike ride or renting a tandem bike
  9. Having a movie night with a homemade cinema setup
  10. Going on a scenic drive to a nearby town or beach
  
  [THOUGHT]
  Now that I have a list of 